# ProjectPegasus

Exam project for the Artificial Intelligence Fundamentals course, a.y. 2023/2024

---

Our code is structured in three main files:
- **Map.py**: These class methods are used for the interactions with the environment: extracting information from cells, rendering the level and making a proper step in the game (applying the actions).
- **Agent.py**: Interacts with Map.py and the knowledge base with two methods: `percept()` and `act()`. 
    - `percept()`: reads information from the environment (using attributes from Map) and populates the knowledge base (using methods from the KBwrapper).
    - `act()`: Queries the knowledge base for the best subtask to execute and launch the related methods.
- **KBwrapper.py**: For the agent, its knowledge base is an instance of this class. Based on pyswip library, its methods are used to manage the "kb.pl" Prolog knowledge base by asserting, retracting and querying the appropriate predicates. 

## Introduction

This project focuses on the task of riding a pony. To do so, the agent:
- Picks up some carrots
- Feeds the pony with carrots, to pacify him and increase his tameness
- Looks for a saddle
- Applies the saddle on the pony and rides it

## Methodologies

Our project is related with the course for these key topics:
- Search algorithm A*
- Prolog Knowledge Base
- explore di davideb?

### Subtasks and Interrupts

We decided to implement an intelligent system for decision making based on interruptions. 
In the case the agent has determined that the best action to perform is to undertake a certain subtask, an interruption notifies to the agent that the premises to continue that subtask are no longer valid and so a subtask switch is needed. 

## Assessment

## Conclusions

## Appendix